* https://matplotlib.org/stable/tutorials/colors/colormapnorms.html
* https://matplotlib.org/stable/tutorials/colors/colormaps.html

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as  sns
import matplotlib as mpl
import datetime
from os.path import join, exists
from typing import List, Tuple, Dict
from datetime import datetime, timedelta
from matplotlib.ticker import FormatStrFormatter, FuncFormatter
from matplotlib import cm
import matplotlib
import matplotlib.colors as colors
import matplotlib.cbook as cbook

In [ ]:
plt.close('all')

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%autosave 300

#### after save to py, what to modify

* delete the above block
* set the following block
* change the indent (1 tabs) and modify the loop


In [ ]:
vars = ['u', 'v', 't', 'kinetic', 'stab', 'heatf', 'momf', 'vort']
vars = ['u', 'v', 't', 'kinetic',  'stab']

plot_on_nb = True
height_lat_Tm_plot = True
height_lat_plot = True
height_lon_plot = True
lat_lon_plot = True
composite_plot = True
composite_lat_lon_plot = True
composite_height_lon_plot = True

sdate_read='0005020100'
edate_read='0005091518'
dh_data = 6

# time average

sdate_Tm='0005020100'
edate_Tm='0005091518'

# time evolution
sdate_plot='0005060318'
edate_plot='0005060318'
dh_plot = 6

# composite
# should be later thatn 000501010 because I don't have solar position for 0005010100
sdate_comp = '0005020100'   
edate_comp = '0005091518'


#sdate_read='0005020100'
#edate_read='0005020106'
#sdate_Tm='0005020100'
#edate_Tm='0005020106'
#sdate_plot='0005020100'
#edate_plot='0005020106'
#sdate_comp = '0005020100'   
#edate_comp = '0005020106'

#path = '/data_jliang01/jliang/venus/bred-qt-NorandomBV-norm40-80km'
#path = '/data_jliang01/jliang/venus/bred-qz'
#path = '/data_jliang01/jliang/venus/bred-qz-NorandomBV-totalenergy'
#path = '/data_jliang01/jliang/venus/bred-qt-NorandomBV-totalenergy'

#path = '/data_jliang01/jliang/venus/bred-qz-NorandomBV-te-t2'
path = '/data_jliang01/jliang/venus/bred-qt-NorandomBV-te-t2'


#path_cont = '/data_jliang01/jliang/venus/spinup-t42-qz'
path_cont = '/data_jliang01/jliang/venus/spinup-t42-qt'

exps = glob.glob(path + '/' + 'rnorm10-rhr06-set001')
path_out_suffix = 'plot2'

In [ ]:
def generate_time_labels(sdate: str, edate: str, dm: int, dday: int, dh: int):   
    yy = int(sdate[:4])
    mm = int(sdate[4:6])
    dd = int(sdate[6:8])
    hh = int(sdate[8:10])
    sdate_int = int(sdate)
    edate_int = int(edate)
    cdate_int = sdate_int
    cdate = sdate
    cdate_list = []
    while cdate_int <= edate_int:
        cdate_list.append(cdate)
        mm = mm + dm
        dd = dd + dday
        hh = hh + dh
        if hh >= 24:
            hh = hh - 24
            dd = dd + 1
            if dd > 30:
                dd = dd - 30
                mm = mm + 1
                if mm > 12:
                    mm = 1
                    yy = yy + 1
        cdate = str(yy).zfill(4) + str(mm).zfill(2) + str(dd).zfill(2) + str(hh).zfill(2)
        cdate_int = int(cdate)
    cdate_list = pd.Series(cdate_list)
    return cdate_list

In [ ]:
time_labels = generate_time_labels(sdate_read, edate_read, dm=0, dday=0, dh=dh_data)
time_labels

In [ ]:
print('how many 6 hours', len(time_labels))
print('how many days', len(time_labels)/4)

In [ ]:
# plot these experiments
print(len(exps))
print(exps)

In [ ]:
size=22
params = {'figure.figsize': (20,8),
          'font.weight': 'light',
          'legend.fontsize': size,
          'axes.labelsize': size*1.2,
          'axes.titlesize': size * 1.2,
          'figure.titlesize': size * 1.2,
          'xtick.labelsize': size*1.1,
          'ytick.labelsize': size*1.3,
          'axes.titlepad': 0,
          }
plt.rcParams.update(params)
plt.rcParams['axes.facecolor'] = 'none'

In [ ]:
def find_rn_rt(s: str):
    """
    find the rescaling norm and the interval from the folder name
    """
    i1 = s.find('rnorm')
    i2 = s.find('rhr')
    rn = float(s[i1+5: i2-1])
    i3 = s.find('set')
    rhr = int(s[i2+3: i3-1])
    set_id = int(s[i3+3: i3+6])
    i4 = s.find('after')
    folder = s[i1: i4-1]
    exp = s[: i4-1]
    return rn, rhr, set_id, folder, exp

In [ ]:
def read_nc_to_dict(nc_file: str, var_names: List) -> Dict:
    """
    nc_file: the netcdf file
    columns: the name of the variable in this nc file
    """
    # print('read variables')
    xa = xr.open_dataset(nc_file)
    var_d = {}
    for var in var_names:
        var_d[var] = np.array(xa[var])
    return var_d

In [ ]:
def set_ticks(data: float, sv: float, ev: float, intv: int, round_digit=0):
    """ Return the ticks and the tick labelof the height
    data: array contains the tick labels
    sv: start label 
    ev: end label, the end label is included
    intv: interval   
    round_digit: number of digits 
    """
    si = np.argmin(np.abs(data-sv))
    ei = np.argmin(np.abs(data-ev))
    label = data[si: ei+1][:: intv]
    label = np.round(label, round_digit)
    si_new = 0
    ei_new = ei +1 - si 
    ticks = np.arange(si_new, ei_new, intv)
    return ticks, label, si, ei

## Read the data

#### (1) read the solar position

In [ ]:
print('read the data \n')
qsol = pd.read_csv('/data_jliang01/jliang/venus/qsol/qsol.csv',converters={'time': lambda x: str(x)})
qsol.index = qsol['time']
qsol.drop(columns=['time'], inplace=True)
qsol

In [ ]:
# grid position along the super rotation
"""
sup_composition = False
sdate = '0005020100'   
edate = '0006010100'
dt = 6
time_labels = generate_time_labels(sdate, edate, 0, 0, 6)

sup_i = pd.Series(np.zeros(len(time_labels), dtype=int), index=time_labels)
sup_i[0] = 64
for i, time in enumerate(time_labels[:-1]):
    sup_i[i+1] = sup_i[i] + 8
    if sup_i[i+1]== 128:
        sup_i[i+1] = 0
"""   

#### (2) read the forecast data

In [ ]:
"""
#? loop over many experiments. 
os.chdir(path)
for rnorm in [237]: #[5, 10, 20, 50, 100, 200, 999]:
    for rhr in [12]: #[12, 24, 48, 120, 240]:
        for set_c in range(0, 1):
            folder = 'rnorm' + str(rnorm) + '-' + 'rhr' + str(rhr) + '-set' + str(set_c).zfill(3)
            if not os.path.exists(folder):
                print('folder is not there')
                continue
            else:
                print('plot for this folder: ', folder)  
"""

In [ ]:
for folder in exps:
    print(folder)
    path_out =  folder + '-' + path_out_suffix
    #print('output the plots to this folder :', path_out)
    if not exists(path_out):
        os.makedirs(path_out)
        
    rnorm, rhr, set_id, _, _  = find_rn_rt(os.path.basename(folder))  
    print('rnorm', rnorm, 'rhr', rhr)
    print('path_out', path_out)

In [ ]:
# get the attribute of the data

files1 = pd.Series(np.sort(glob.glob(folder + '/anal/*.nc' )))

#files2 = pd.Series(np.sort(glob.glob(path_cont + '*.nc' )))
#filenames = files1.apply(lambda x: os.path.basename(x)[:10])
#num_f = len(filenames)

f0 = files1[0]
xa0 =  xr.open_dataset(f0)
levs = np.array(xa0['lev'])
lats = np.array(xa0['lat'])
lons = np.array(xa0['lon'])
lat_cos = np.cos(lats / 180 * 3.1416)
lat_cos_sqrt = np.sqrt(lat_cos)
lat_cos_4d = lat_cos[np.newaxis, np.newaxis, :,  np.newaxis]
lat_cos_sqrt_4d = lat_cos_sqrt[np.newaxis, np.newaxis, :,  np.newaxis]

nlon = len(lons)
nlat = len(lats)
nlev = len(levs)
print('nlon', nlon, 'nlat', nlat, 'nlev', nlev)

In [ ]:
# sigma level
sigma = np.array([ 0.939209707652, 0.824800040009, 0.722071094351, 0.630071458909, 0.547906620089, 
         0.474736813045, 0.409775074481, 0.352285034343, 0.301578978584, 0.257015834561, 
          0.217999185328, 0.183975458860, 0.154431729160, 0.128894025505, 0.106925587127, 
          0.881248715475e-01, 0.721238162892e-01, 0.585864981132e-01, 0.472148614674e-01, 0.377477164306e-01, 
          0.299443276525e-01, 0.235765653669e-01, 0.184314409939e-01, 0.143074291149e-01, 0.110170654275e-01, 
          0.840076632025e-02, 0.633005828108e-02, 0.470135325860e-02, 0.343203787341e-02, 0.245616317258e-02, 
          0.171891574566e-02, 0.117417455248e-02, 0.782341977019e-03, 0.508462352658e-03, 0.322558057495e-03, 
          0.200015476611e-03, 0.121511572690e-03, 0.725521621617e-04, 0.427478113682e-04, 0.249732323235e-04, 
          0.145363675087e-04, 0.845662546416e-05, 0.491969484866e-05, 0.286206373781e-05, 0.166502382355e-05, 
          0.968638276239e-06, 0.563511495180e-06, 0.327826385521e-06, 0.190715075915e-06, 0.110949707305e-06, 
          0.645457011307e-07, 0.375498744167e-07, 0.218448791431e-07, 0.127083964758e-07, 0.739319081188e-08, 
          0.430103546397e-08, 0.250215439937e-08, 0.145564437511e-08, 0.846830529975e-09, 0.251897477798e-09 ])

si_h = 20
ei_h = 40
sigma_diff = np.zeros(nlev)
sigma_diff[:nlev-1] = sigma[:-1] - sigma[1:]
sigma_diff[nlev-1] = sigma[nlev-1]

# get differnet dimension versios of the sigma diff
sigma_diff_2d = sigma_diff[:, np.newaxis]
sigma_diff_3d = sigma_diff[:, np.newaxis, np.newaxis]
sigma_diff_4d = sigma_diff[np.newaxis, :,  np.newaxis, np.newaxis]

# square root of sigma
sigma_diff_sqrt = np.sqrt(sigma_diff)
sigma_diff_sqrt_2d =  sigma_diff_sqrt[:, np.newaxis]
sigma_diff_sqrt_3d =  sigma_diff_sqrt[:, np.newaxis, np.newaxis]
sigma_diff_sqrt_4d =  sigma_diff_sqrt[np.newaxis, :,  np.newaxis, np.newaxis]

# set the scale factor to scale the data
s_v = sigma_diff_sqrt_4d
s_h = lat_cos_sqrt_4d
s_hv = sigma_diff_sqrt_4d * lat_cos_sqrt_4d
s2_v = sigma_diff_4d
s2_h = lat_cos_4d
s2_hv = sigma_diff_4d * lat_cos_4d

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(sigma[si_h: ei_h], range(si_h, ei_h), label='sigma')
plt.plot(sigma_diff[si_h: ei_h], range(si_h, ei_h), label='sigma-diff')
plt.legend()
plt.ylabel('Height (km)')

In [ ]:
# read the data one by one according to the time labels

start_time = datetime.now()
var_names = ['u', 'v', 't', 'ps']
cont = {}
pert = {}
last_index = 0
for i, time in enumerate(time_labels):
    print('read ', time)
    f = folder + '/anal/' + time + '.nc'
    if  os.path.exists(f):
        cont[time] = read_nc_to_dict(path_cont + '/' + time + '.nc', var_names)
        pert[time] = read_nc_to_dict(f, var_names)    
        last_index = i
    else:
        print(f, ' is not here')

    if last_index == 0:
        continue
time_labels = time_labels[:last_index+1]
end_time = datetime.now()
print(end_time - start_time)

In [ ]:
len(time_labels)

In [ ]:
# combine the time series

gs = {}
anal = {}
for var in var_names:
    print(var)
    gs[var] = []
    anal[var] = []
    for time in time_labels:
        gs[var].append(cont[time][var])
        anal[var].append(pert[time][var])       
    gs[var] = np.concatenate(gs[var], axis=0)
    anal[var] = np.concatenate(anal[var], axis=0)
    print(gs[var].shape)

## setting for plotting

#### set  coordinate

In [ ]:
print('setting for plotting \n')

In [ ]:
# for values with negative and positve numbers
colormap_div = 'bwr'  
#colormap_div = cm.coolwarm
# for positive numbers
colormap_seq = 'Reds'

In [ ]:
#  height coordinate in the data

print('number of layers', nlev)       
heights = np.arange(1, 120, 2)
print(heights)
print(len(heights))

# set height you want to plot
#?
sh = 55   
eh = 100   # km

sh = 0   
eh = 90   # km

sh = 30
eh = 100   # km

intv = 5  # levels

h_ticks, h_ticklabels, si_h, ei_h = set_ticks(heights, sh, eh, intv)
print('h_ticks ', h_ticks)
print('h_ticklabels' , h_ticklabels)
print('si_h ', si_h, 'ei_h ', ei_h)

# only plot even number
h_ticks = h_ticks + 0.5
h_ticklabels = h_ticklabels + 1
print('h_ticks ', h_ticks)
print('h_ticklabels' , h_ticklabels)

#### set xy labels

In [ ]:
lon_ticks, lon_ticklabels, si_lon, ei_lon = set_ticks(lons, 0, 360, 20, round_digit=1)
print('si ', si_lon, 'ei ', ei_lon)
print('lon_ticks \n', lon_ticks)
print('lon_ticklabels \n', lon_ticklabels)

lat_ticks, lat_ticklabels, si_lat, ei_lat = set_ticks(lats, -90, 90, 4, round_digit=1)
print('si_lat ', si_lat, 'ei_lat ', ei_lat)
print('lat_ticks \n', lat_ticks)
print('lat_ticklabels \n', lat_ticklabels)

## process the data

#### calculate pressure,  density

In [ ]:
R = 191.4
for i, tmp in enumerate([gs, anal]):  
    t = tmp['t']    # dims:  (121, 60, 64, 128)
    ps = tmp['ps']  # dims: (121, 1, 64, 128)
    dims = t.shape
    p = np.zeros(dims)
    for k in range(dims[1]):
        p[:, k] = ps[:, 0] * sigma[k]
    rho = p / (R * t)
    
    # average density profile
    print(rho.shape) # dims: (121, 60, 64, 128)
    rho_pf = rho.mean(axis=(0, 2, 3), dtype=np.float64) # dims (60,)
    
    # average density  at 70km
    rho_pf_70km = rho_pf[34]
    
    # ratio to the density at 70km
    rho_ratio = rho_pf / rho_pf_70km  
    rho_ratio_sqrt = np.sqrt(rho_ratio)

    # get different dimension versions.
    rho_ratio_2d = rho_ratio[:, np.newaxis]
    rho_ratio_3d = rho_ratio[:, np.newaxis, np.newaxis]
    rho_ratio_4d = rho_ratio[np.newaxis, :,  np.newaxis, np.newaxis]
    rho_ratio_sqrt_2d =  rho_ratio_sqrt[:, np.newaxis]
    rho_ratio_sqrt_3d =  rho_ratio_sqrt[:, np.newaxis, np.newaxis]
    rho_ratio_sqrt_4d =  rho_ratio_sqrt[np.newaxis, :,  np.newaxis, np.newaxis]

    if i == 0:
        gs['p'] = p
        gs['rho'] = rho
        rho_ratio_gs = rho_ratio
        rho_ratio_2d_gs = rho_ratio_2d
        rho_ratio_3d_gs = rho_ratio_3d
        rho_ratio_4d_gs = rho_ratio_4d
        rho_ratio_sqrt_gs = rho_ratio_sqrt
        rho_ratio_sqrt_2d_gs = rho_ratio_sqrt_2d
        rho_ratio_sqrt_3d_gs = rho_ratio_sqrt_3d
        rho_ratio_sqrt_4d_gs = rho_ratio_sqrt_4d
                  
    else:
        anal['p']= p
        anal['rho'] = rho
        rho_ratio_anal = rho_ratio
        rho_ratio_2d_anal = rho_ratio_2d
        rho_ratio_3d_anal = rho_ratio_3d
        rho_ratio_4d_anal = rho_ratio_4d
        rho_ratio_sqrt_anal = rho_ratio_sqrt
        rho_ratio_sqrt_2d_anal = rho_ratio_sqrt_2d
        rho_ratio_sqrt_3d_anal = rho_ratio_sqrt_3d
        rho_ratio_sqrt_4d_anal = rho_ratio_sqrt_4d       

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(rho_pf[si_h: ei_h+1], heights[si_h: ei_h+1], label='density ')
plt.legend()
plt.ylabel('Height (km)')

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(rho_ratio[si_h: ei_h+1], heights[si_h: ei_h+1], label='density ratio')
plt.plot(rho_ratio_sqrt[si_h: ei_h+1], heights[si_h: ei_h+1], label='sqrt of density ratio')
plt.legend()
plt.ylabel('Height (km)')

In [ ]:
# compare density and sigma difference profiles. standardize them before the comparison.
a = rho_ratio[si_h: ei_h+1]
b = sigma_diff[si_h: ei_h+1] 
a =  a / a.max() 
b =  b / b.max()

c = rho_ratio_sqrt[si_h: ei_h+1]
d = sigma_diff_sqrt[si_h: ei_h+1] 
c =  c / c.max() 
d =  d / d.max()

e = sigma[si_h: ei_h+1] 
e = e / c.max()

plt.figure(figsize=(10,5))

plt.plot(a, heights[si_h: ei_h+1], label='density ratio')
plt.plot(b, heights[si_h: ei_h+1], label='sigma diff')
plt.plot(c, heights[si_h: ei_h+1], label='density ratio sqrt')
plt.plot(d, heights[si_h: ei_h+1], label='sigma diff sqrt')
plt.plot(e, heights[si_h: ei_h+1], label='sigma')

plt.legend()
plt.ylabel('Height (km)')

#### calculate kinetic energy density

In [ ]:
if 'kinetic' in vars: 
    for i, typ in enumerate([gs, anal]):
        u = typ['u']
        u_m = u.mean(axis=3, keepdims=True, dtype=np.float64)
        u_d =u - u_m

        v = typ['v']
        v_m = v.mean(axis=3, keepdims=True, dtype=np.float64)
        v_d =v - v_m
       
        typ['kinetic'] = 0.5 * typ['rho'] * (u_d ** 2  +  v_d ** 2)

#### calculate  heat flux

In [ ]:
if 'heatf' in vars:
    for i, typ in enumerate([gs, anal]):
        t = typ['t']
        t_m = t.mean(axis=3, keepdims=True, dtype=np.float64)
        t_d = t - t_m

        v = typ['v']
        v_m = v.mean(axis=3, keepdims=True, dtype=np.float64)
        v_d =v - v_m
        
        typ['heatf'] = t_d * v_d 

In [ ]:
anal.keys()

#### calculate momemtum flux

In [ ]:
if 'momf' in vars: 
    for i, typ in enumerate([gs, anal]):
        u = typ['u']
        u_m = u.mean(axis=3, keepdims=True, dtype=np.float64)
        u_d =u - u_m

        v = typ['v']
        v_m = v.mean(axis=3, keepdims=True, dtype=np.float64)
        v_d =v - v_m

        typ['momf'] = u_d * v_d  

####  calculate the static stability

In [ ]:
if 'stab' in vars:
    for typ in [gs, anal]:
        typ['stab'] = np.zeros(typ['t'].shape)
        # python exclude the last index, so nlev-1 is not included
        typ['stab'][:, :nlev-1, :, :] = (typ['t'][:, 1: , :, : ] - typ['t'][:, : nlev-1, :, : ] ) / 2 + 8.87

#### calculate vorticity

In [ ]:
if 'vort' in vars:
    radius_venus = 6051800 
    radius = radius_venus + heights * 1000
    circles = 2 * np.pi * radius
    dlat_3d = np.zeros((nlev, nlat, nlon))
    dlon_3d = np.zeros((nlev, nlat, nlon))
    for j in range(nlat):
        for i in range(nlon):
            dlat_3d[:, j, i] = circles / (2 * nlat)    
            dlon_3d[:, j, i] = circles / nlon * lat_cos[j]

    for typ in [gs, anal]:
        u = typ['u']
        u_m = u.mean(axis=3, keepdims=True, dtype=np.float64)
        u_dev =u - u_m

        v = typ['v']
        v_m = v.mean(axis=3, keepdims=True, dtype=np.float64)
        v_dev =v - v_m

        v_dev_diff = np.zeros(u.shape)
        v_dev_diff[:, :, :, :-1] = v_dev[:, :, :, 1:] - v_dev[:, :, :, :-1]
        v_dev_diff[:, :, :, nlon-1] = v_dev[:, :, :, 0] - v_dev[:, :, :, nlon-1] 

        u_dev_diff = np.zeros(u.shape)
        u_dev_diff[:, :,  :-1, :] = u_dev[:, :,  1:, :] - u_dev[:, :, :-1, :]
        # TODO this may be wrong, north and south does not form a circle?
        u_dev_diff[:, :, nlat-1, :] = u_dev[:, :, 0, :] - u_dev[:, :, nlat-1, :] 

        typ['vort']  = v_dev_diff / dlon_3d - u_dev_diff / dlat_3d 

#### time average

In [ ]:
time_labels

In [ ]:
# set the time range for the averaging 
times_Tm = generate_time_labels(sdate_Tm, edate_Tm, dm=0, dday=0, dh=6)

# get the index in the original time that are inside the time ragne for the averaging.
index = time_labels[time_labels.isin(times_Tm)].index
print(index)

# start and end index for taking the time averaging
si_Tm = index[0]
ei_Tm = index[-1]

#### calculate the difference

In [ ]:
gs.keys()

In [ ]:
diff = {}
for var in vars:
    diff[var] = anal[var] - gs[var]

#### calculate all diagnostic variables

In [ ]:
sigma_diff_4d.shape

In [ ]:
lat_cos_4d.shape

In [ ]:
for i, typ in enumerate([gs, anal, diff]):
    for var in vars:
        print(var)
        t = typ[var]
        
        # zonal mean
        t_Zm = t.mean(axis=3, keepdims=True, dtype=np.float64)
        typ[var+'_Zm'] = t_Zm
        
        # zonal and temperal mean
        t_Zm_Tm = t_Zm[si_Tm: ei_Tm+1].mean(axis=0, keepdims=True,  dtype=np.float64)  
        typ[var+'_Zm_Tm'] = t_Zm_Tm

        # temperal mean
        t_Tm = t[si_Tm: ei_Tm+1].mean(axis=0, keepdims=True , dtype=np.float64) 
        typ[var+'_Tm'] = t_Tm
        
        # deviation from the zonal mean
        t_ZmDev = t - t_Zm
        t_ZmDev_abs = np.abs(t_ZmDev) 
        typ[var+'_ZmDev'] = t_ZmDev
        typ[var+'_ZmDev_abs'] = t_ZmDev_abs
        
        # square of the deviation
        t_ZmDev_sq = t_ZmDev ** 2
        typ[var+'_ZmDev_sq'] = t_ZmDev_sq
        
        # zonal mean of the square of the deviation
        t_ZmDev_sq_Zm = t_ZmDev_sq.mean(axis=3, keepdims=True, dtype=np.float64)   
        typ[var+'_ZmDev_sq_Zm'] = t_ZmDev_sq_Zm
        
        # root of the zonal mean of the square of the deviation
        t_ZmDev_sq_Zm_sqrt = np.sqrt(t_ZmDev_sq_Zm)
        typ[var+'_ZmDev_sq_Zm_sqrt'] = t_ZmDev_sq_Zm_sqrt
        
        # zonal and tempeoral mean of the square of the deviation
        t_ZmDev_sq_Zm_Tm = t_ZmDev_sq_Zm[si_Tm: ei_Tm+1].mean(axis=0, keepdims=True, dtype=np.float64) 
        typ[var+'_ZmDev_sq_Zm_Tm'] = t_ZmDev_sq_Zm_Tm
        
        # root of the zonal and temporal mean of the square of the deviation
        t_ZmDev_sq_Zm_Tm_sqrt = np.sqrt(t_ZmDev_sq_Zm_Tm)
        typ[var+'_ZmDev_sq_Zm_Tm_sqrt'] = t_ZmDev_sq_Zm_Tm_sqrt
        
        # meridional deviation for temperature
        if ((i == 0) | (i == 1)) & (var == 't'):
            # meridional deviation of the zonal mean
            t_Zm_Mm = t_Zm.mean(axis=2, keepdims=True, dtype=np.float64)
            t_Zm_MmDev = t_Zm - t_Zm_Mm
            typ[var+'_Zm_MmDev'] = t_Zm_MmDev
            
            # meridional deviation of the zonal and time mean
            t_Zm_Tm_Mm = t_Zm_Tm.mean(axis=2, keepdims=True, dtype=np.float64)
            t_Zm_Tm_MmDev = t_Zm_Tm - t_Zm_Tm_Mm   
            typ[var+'_Zm_Tm_MmDev'] = t_Zm_Tm_MmDev
            
        # only for the  bred vector
        if i == 2:
            t_abs = np.abs(t)
            typ[var+'_abs'] = t_abs
            
            t_sq = t ** 2
            typ[var+'_sq'] = t_sq
            
            t_sq_Zm = t_sq.mean(axis=3, keepdims=True, dtype=np.float64)
            typ[var+'_sq_Zm'] = t_sq_Zm
            
            t_sq_Zm_sqrt = np.sqrt(t_sq_Zm)
            typ[var+'_sq_Zm_sqrt'] = t_sq_Zm_sqrt
            
            t_sq_Zm_Tm = t_sq_Zm[si_Tm: ei_Tm+1].mean(axis=0, keepdims=True, dtype=np.float64)  
            typ[var+'_sq_Zm_Tm'] = t_sq_Zm_Tm
            
            t_sq_Zm_Tm_sqrt = np.sqrt(t_sq_Zm_Tm)
            typ[var+'_sq_Zm_Tm_sqrt'] = t_sq_Zm_Tm_sqrt


        ######
        # scale the data
        ########
        t_s_v = t * s_v 
        t_s_h = t * s_h
        t_s_hv = t * s_hv  
        
        typ[var+'_s_v'] = t_s_v
        typ[var+'_s_h'] = t_s_h
        typ[var+'_s_hv'] = t_s_hv
        
        t_s2_v = t * s2_v 
        typ[var+'_s2_v'] = t_s2_v
        
        t_Zm_s_hv = t_Zm * s_hv
        t_Zm_Tm_s_hv = t_Zm_Tm * s_hv
        t_Zm_s2_hv = t_Zm * s2_hv
        t_Zm_Tm_s2_hv = t_Zm_Tm * s2_hv
        
        typ[var+'_Zm_s_hv'] = t_Zm_s_hv 
        typ[var+'_Zm_Tm_s_hv'] = t_Zm_Tm_s_hv
        typ[var+'_Zm_s2_hv'] = t_Zm_s2_hv
        typ[var+'_Zm_Tm_s2_hv'] = t_Zm_Tm_s2_hv

        
        t_ZmDev_s_v = t_ZmDev * s_v
        t_ZmDev_abs_s_v = t_ZmDev_abs * s_v
        
        typ[var+'_ZmDev_s_v'] = t_ZmDev_s_v
        typ[var+'_ZmDev_abs_s_v'] = t_ZmDev_abs_s_v
        
        
        # rms
        t_ZmDev_sq_Zm_sqrt_s_v = t_ZmDev_sq_Zm_sqrt * s_v
        t_ZmDev_sq_Zm_sqrt_s_hv = t_ZmDev_sq_Zm_sqrt * s_hv
        t_ZmDev_sq_Zm_Tm_sqrt_s_v = t_ZmDev_sq_Zm_Tm_sqrt * s_v
        t_ZmDev_sq_Zm_Tm_sqrt_s_hv = t_ZmDev_sq_Zm_Tm_sqrt * s_hv
        
        typ[var+'_ZmDev_sq_Zm_sqrt_s_v'] = t_ZmDev_sq_Zm_sqrt_s_v
        typ[var+'_ZmDev_sq_Zm_sqrt_s_hv'] = t_ZmDev_sq_Zm_sqrt_s_hv
        typ[var+'_ZmDev_sq_Zm_Tm_sqrt_s_v'] = t_ZmDev_sq_Zm_Tm_sqrt_s_v
        typ[var+'_ZmDev_sq_Zm_Tm_sqrt_s_hv'] = t_ZmDev_sq_Zm_Tm_sqrt_s_hv
        
        # squre
        t_ZmDev_sq_Zm_s2_v = t_ZmDev_sq_Zm * s2_v
        t_ZmDev_sq_Zm_s2_hv = t_ZmDev_sq_Zm * s2_hv
        t_ZmDev_sq_Zm_Tm_s2_v = t_ZmDev_sq_Zm_Tm * s2_v
        t_ZmDev_sq_Zm_Tm_s2_hv = t_ZmDev_sq_Zm_Tm * s2_hv
        
        typ[var+'_ZmDev_sq_Zm_s2_v'] = t_ZmDev_sq_Zm_s2_v
        typ[var+'_ZmDev_sq_Zm_s2_hv'] = t_ZmDev_sq_Zm_s2_hv
        typ[var+'_ZmDev_sq_Zm_Tm_s2_v'] = t_ZmDev_sq_Zm_Tm_s2_v
        typ[var+'_ZmDev_sq_Zm_Tm_s2_hv'] = t_ZmDev_sq_Zm_Tm_s2_hv
        
        # scale the BV
        if i == 2:
            
            t_s_v = t * s_v
            t_abs_s_v = t_abs * s_v
            
            typ[var+'_s_v'] = t_s_v
            typ[var+'_abs_s_v'] = t_abs_s_v
                
            # rms
            t_sq_Zm_sqrt_s_v = t_sq_Zm_sqrt * s_v
            t_sq_Zm_sqrt_s_hv = t_sq_Zm_sqrt * s_hv
            t_sq_Zm_Tm_sqrt_s_v = t_sq_Zm_Tm_sqrt * s_v
            t_sq_Zm_Tm_sqrt_s_hv = t_sq_Zm_Tm_sqrt * s_hv
            
            typ[var+'_sq_Zm_sqrt_s_v'] = t_sq_Zm_sqrt_s_v
            typ[var+'_sq_Zm_sqrt_s_hv'] = t_sq_Zm_sqrt_s_hv
            typ[var+'_sq_Zm_Tm_sqrt_s_v'] = t_sq_Zm_Tm_sqrt_s_v
            typ[var+'_sq_Zm_Tm_sqrt_s_hv'] = t_sq_Zm_Tm_sqrt_s_hv
            
            # squre
            t_sq_Zm_s2_v = t_sq_Zm * s2_v
            t_sq_Zm_s2_hv = t_sq_Zm * s2_hv
            t_sq_Zm_Tm_s2_v = t_sq_Zm_Tm * s2_v
            t_sq_Zm_Tm_s2_hv = t_sq_Zm_Tm * s2_hv
            
            typ[var+'_sq_Zm_s2_v'] = t_sq_Zm_s2_v
            typ[var+'_sq_Zm_s2_hv'] = t_sq_Zm_s2_hv
            typ[var+'_sq_Zm_Tm_s2_v'] = t_sq_Zm_Tm_s2_v
            typ[var+'_sq_Zm_Tm_s2_hv'] = t_sq_Zm_Tm_s2_hv               

### calculate the total energy in the perturbation

In [ ]:
diff.keys()

In [ ]:
du = diff['u']
dv = diff['v']
dt = diff['t']
ps = gs['ps']

In [ ]:
kinet = (du ** 2 + dv ** 2) * ps *  sigma_diff_4d * lat_cos_4d
t2 = 1000 /  737.15 *  dt ** 2 * ps *  sigma_diff_4d * lat_cos_4d
total = kinet + t2

In [ ]:
diff['kinet_Zm'] = kinet.mean(axis=3, keepdims=True, dtype=np.float64)
diff['kinet_Zm_Tm'] = diff['kinet_Zm'][si_Tm: ei_Tm+1].mean(
    axis=0, keepdims=True, dtype=np.float64) 
diff['t2_Zm'] = t2.mean(axis=3, keepdims=True, dtype=np.float64)
diff['t2_Zm_Tm'] = diff['t2_Zm'][si_Tm: ei_Tm+1].mean(axis=0, keepdims=True, dtype=np.float64)  
diff['total_Zm'] = total.mean(axis=3, keepdims=True, dtype=np.float64)
diff['total_Zm_Tm'] = diff['total_Zm'][si_Tm: ei_Tm+1].mean(
    axis=0, keepdims=True, dtype=np.float64)  

## plot

#### time range for plotting the time evolution plot

In [ ]:
time_labels

In [ ]:
time_labels_plot = generate_time_labels(sdate_plot, edate_plot,  dm=0, dday=0, dh=dh_plot)
print(time_labels_plot)

In [ ]:
plot_index = time_labels[time_labels.isin(time_labels_plot)].index
plot_index

#### Height Lat,  time average

In [ ]:
path_out

In [ ]:
if height_lat_Tm_plot:
    start_time = datetime.now()
    
    # if you don't want to plot, need to place a place holder in the array ('', '')

    plot_list = np.array([
    [('u_Zm_Tm', gs), ('u_Zm_Tm', anal), ('u_sq_Zm_Tm_sqrt_s_hv', diff), ('u_sq_Zm_Tm_s2_hv', diff)],
    [('v_Zm_Tm', gs), ('v_Zm_Tm', anal), ('v_sq_Zm_Tm_sqrt_s_hv', diff), ('v_sq_Zm_Tm_s2_hv', diff)],
    [('t_Zm_Tm', gs), ('t_Zm_Tm', anal), ('t_sq_Zm_Tm_sqrt_s_hv', diff), ('t_sq_Zm_Tm_s2_hv', diff)],
    [('u_ZmDev_sq_Zm_Tm_sqrt_s_hv', gs),  ('u_ZmDev_sq_Zm_Tm_s2_hv', anal), ('', ''), ('', '')],
    [('v_ZmDev_sq_Zm_Tm_sqrt_s_hv', gs),  ('v_ZmDev_sq_Zm_Tm_s2_hv', anal), ('', ''), ('', '')],
    [('t_ZmDev_sq_Zm_Tm_sqrt_s_hv', gs),  ('t_ZmDev_sq_Zm_Tm_s2_hv', anal), ('', ''), ('', '')],
    [('t_Zm_Tm_MmDev', gs), ('t_Zm_Tm_MmDev', anal), ('', ''), ('', '')],
    ])
    plot_list = np.array([
        [('u_Zm_Tm', gs), ('u_Zm_Tm', anal), ('u_sq_Zm_Tm_sqrt_s_v', diff), ('u_sq_Zm_Tm_s2_v', diff)],
        [('v_Zm_Tm', gs), ('v_Zm_Tm', anal), ('v_sq_Zm_Tm_sqrt_s_v', diff), ('v_sq_Zm_Tm_s2_v', diff)],
        [('t_Zm_Tm', gs), ('t_Zm_Tm', anal), ('t_sq_Zm_Tm_sqrt_s_v', diff), ('t_sq_Zm_Tm_s2_v', diff)],
        [('u_ZmDev_sq_Zm_Tm_sqrt_s_v', gs),  ('u_ZmDev_sq_Zm_Tm_s2_v', anal), ('', ''), ('', '')],
        [('v_ZmDev_sq_Zm_Tm_sqrt_s_v', gs),  ('v_ZmDev_sq_Zm_Tm_s2_v', anal), ('', ''), ('', '')],
        [('t_ZmDev_sq_Zm_Tm_sqrt_s_v', gs),  ('t_ZmDev_sq_Zm_Tm_s2_v', anal), ('', ''), ('', '')],
        [('t_Zm_Tm_MmDev', gs), ('t_Zm_Tm_MmDev', anal), ('', ''), ('', '')],
    ])   
    
    plot_list = np.array([
    [('u_Zm_Tm', gs), ('u_Zm_Tm', anal), ('u_sq_Zm_Tm_sqrt_s_v', diff), ('kinet_Zm_Tm', diff)],
    [('v_Zm_Tm', gs), ('v_Zm_Tm', anal), ('v_sq_Zm_Tm_sqrt_s_v', diff), ('total_Zm_Tm', diff)],
    [('t_Zm_Tm', gs), ('t_Zm_Tm', anal), ('t_sq_Zm_Tm_sqrt_s_v', diff, diff), ('t2_Zm_Tm', diff)],
    [('u_ZmDev_sq_Zm_Tm_sqrt_s_v', gs),  ('u_ZmDev_sq_Zm_Tm_s2_v', anal), ('', ''), ('', '')],
    [('v_ZmDev_sq_Zm_Tm_sqrt_s_v', gs),  ('v_ZmDev_sq_Zm_Tm_s2_v', anal), ('', ''), ('', '')],
    [('t_ZmDev_sq_Zm_Tm_sqrt_s_v', gs),  ('t_ZmDev_sq_Zm_Tm_s2_v', anal), ('', ''), ('', '')],
    [('t_Zm_Tm_MmDev', gs), ('t_Zm_Tm_MmDev', anal), ('', ''), ('', '')],
    [('kinetic_Zm_Tm', gs), ('kinetic_Zm_Tm', anal), ('', ''), ('', '')],
    [('stab_Zm_Tm', gs), ('stab_Zm_Tm', anal), ('stab_Zm_Tm', diff), ('', '')]
    ])
    #        

        #[('stab_Zm_Tm', gs), ('stab_Zm_Tm', anal), ('stab_Zm_Tm', diff), ('', '')], 
        #[('heatf_Zm_Tm_s2_hv', gs), ('heatf_Zm_Tm_s2_hv', anal), ('heatf_Zm_Tm_s2_hv', diff), ('', '')],
        #[('momf_Zm_Tm_s2_hv', gs), ('momf_Zm_Tm_s2_hv', anal), ('momf_Zm_Tm_s2_hv', diff), ('', '')],   
        
    typ_name = ['gs', 'anal', 'diff', 'diff']
    
    cbar_format = np.array([
        ['%.1f', '%.1f', '%.2f', '%.1e', '%.2f'], 
        ['%.1f', '%.1f', '%.2f', '%.1e', '%.2f'],
        ['%.1f', '%.1f', '%.2f', '%.1e', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.1e', '%.1e', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.1e', '%.1e', '%.1e', '%.2f', '%.1f'],
        ['%.3f', '%.3f', '%.1e', '%.2f', '%.1f']
    ])

    cbar_skip = np.array([
        [2, 2, 2, 2, 999, 999], 
        [2, 2, 2, 2, 999, 999],
        [2, 2, 999, 2, 999, 999],
        [2, 2, 2, 2, 999, 999],
        [2, 2, 2, 2, 999, 999],
        [999, 2, 2, 2, 999, 999],
        [2, 2, 2, 2, 999, 999],
        [2, 2, 2, 2, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999]        
    ])


    cmap = np.array([
        [colormap_div, colormap_div,  colormap_seq, colormap_seq, colormap_seq], 
        [colormap_div, colormap_div,  colormap_seq, colormap_seq, colormap_seq],
        [colormap_div, colormap_div,  colormap_seq, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_seq, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_seq, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_seq, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_seq, colormap_seq, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_div, colormap_seq, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_div, colormap_seq, colormap_seq],
    ])

    set_levels = np.array([
        [False, False, True, False, False],
        [False, False, True, False, False],
        [False, False, True, False, False],
        [True, False, False, False, False],        
        [True, False, False, False, False],
        [True, False, False, False, False],
        [False, False, False, False, False],
        [False, False, False, False, False], 
        [False, False, False, False, False],       
    ])

    vort_range = np.arange(-1e-4, 1.2e-4, 0.2e-4)
    vort_range = np.where(abs(vort_range) < 2e-20, 0.0, vort_range)
    conlevels = np.array([
        [np.arange(-10, 150, 10), np.arange(-50, 150, 10), np.arange(0, 0.5, 0.05), np.arange(0, 14, 1)],
        [np.arange(-10, 12, 2), np.arange(-25, 30, 5), np.arange(0, 0.5, 0.05), np.arange(0, 14, 1)],
        [np.arange(-30, 35,  5), np.arange(-30, 35,  5), np.arange(0, 0.11, 0.01), np.arange(0, 3, 0.2)],
        [np.arange(0, 0.5, 0.05), np.arange(-30, 35,  5), np.arange(0, 3, 0.2), np.arange(0, 3, 0.2)],
        [np.arange(0, 0.5, 0.05), np.arange(0, 14, 1), '', ''], 
        [np.arange(0, 0.11, 0.01), np.arange(0, 14, 1), '', ''], 
        [np.arange(0, 3, 0.2), np.arange(0, 3, 0.2), '', ''] ,    
        [np.arange(0, 22, 2), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1)],
        [np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1)],
        [np.arange(-12, 13,  1), np.arange(-12, 13,  1), np.arange(-5, 5.5, 0.5), np.arange(0, 1600, 100)],
        [np.arange(-45, 50, 5), np.arange(-45, 50, 5), np.arange(-10, 11, 1), np.arange(0, 1e4, 500)],
        
    ])

    print('Height Lat time mean \n')
    pdf_file = path_out +'/height-lat-Tm-' + sdate_Tm + '-' + edate_Tm + '.pdf'
    with PdfPages(pdf_file) as pdf: 
        fig, axes = plt.subplots(len(plot_list), len(plot_list[0]), figsize=(20 * len(plot_list[0]), 20*len(plot_list)), sharex=False)
        for i in range(len(plot_list)):
            for j in range(len(plot_list[i])):
                ax = axes[i, j]  
                var = plot_list[i, j][0]
                typ = plot_list[i, j][1]
                if var == '':
                    continue
                data = typ[var][0, si_h: ei_h+1, :, 0]
                if set_levels[i, j]:
                    fills = ax.contourf(data, cmap=cmap[i, j], levels=conlevels[i, j], extend='max')
                else:
                    if cmap[i, j] == colormap_div:
                        fills = ax.contourf(data, cmap=cmap[i, j], norm=colors.CenteredNorm(), extend='both')
                    else:
                        fills = ax.contourf(data, cmap=cmap[i, j], extend='max' ) # norm=colors.LogNorm(vmin=data.min(), vmax=data.max())
                cbar = fig.colorbar(fills, orientation="horizontal", ax=ax,  format=cbar_format[i, j], pad=0.18)
                cbar.ax.tick_params(labelsize=50)
                if cbar_skip[i, j] != 999:
                    for label in cbar.ax.xaxis.get_ticklabels()[::cbar_skip[i, j]]:
                        label.set_visible(False) 
                        
                # overlay other variables
                if var == 'u_ZmDev_sq_Zm_Tm_sqrt_s_v':
                    CS = ax.contour(gs['u_Zm_Tm'][0, si_h: ei_h+1, :, 0], 
                               colors='Black', linewidths=3) #, levels=np.arange(-20, 120, 20))
                    ax.clabel(CS, inline=1, fontsize=45)
                if var == 'v_ZmDev_sq_Zm_Tm_sqrt_s_v':
                    CS = ax.contour(gs['v_Zm_Tm'][0, si_h: ei_h+1, :, 0], 
                               colors='Black', linewidths=3, levels=np.arange(-20, 20, 5))
                    ax.clabel(CS, inline=1, fontsize=45, colors='Blue', levels= [-15, 0, 15])
                if var == 't_ZmDev_sq_Zm_Tm_sqrt_s_v':
                    CS = ax.contour(gs['t_Zm_Tm_MmDev'][0, si_h: ei_h+1, :, 0], 
                               colors='Black', linewidths=3) #, levels=np.arange(-20, 120, 20))
                    ax.clabel(CS, inline=1, fontsize=45) 
                  

                        
                ax.set_title( typ_name[j] + ' \n ' + var, pad = 40, fontsize=55)     
                ax.set_xlabel('Latitude', fontsize=55)
                ax.set_ylabel('Height (km)', fontsize=55)
                ax.set_xticks(lat_ticks)
                ax.set_xticklabels(lat_ticklabels, rotation=45, fontsize=55)        
                ax.set_yticks(h_ticks)
                ax.set_yticklabels(h_ticklabels, fontsize=55)                 
        fig.tight_layout()
        # plt.legend(loc='upper left',  bbox_to_anchor=(1, 0.5))
        fig.suptitle('rnorm-' + str(rnorm) + '-rhr-' + str(rhr) + ' \n'+
                     'Zm  ' + 'time Average ' + str(sdate_Tm) + '-' + str(edate_Tm) )
        fig.subplots_adjust(top=0.95)
        pdf.savefig(bbox_inches = 'tight') 
        if plot_on_nb: 
            plt.show()
        plt.close(fig)
        
end_time = datetime.now()
print(end_time - start_time)

#### Height Lat time evolution

In [ ]:
if height_lat_plot:
    start_time = datetime.now()
    
    # if you don't want to plot, need to place a place holder in the array ('', '')

    plot_list = np.array([
        [('u', gs), ('u', anal), ('u_sq_Zm_sqrt_s_hv', diff), ('u_sq_Zm_s2_hv', diff)],
        [('v', gs), ('v', anal), ('v_sq_Zm_sqrt_s_hv', diff), ('v_sq_Zm_s2_hv', diff)],
        [('t', gs), ('t', anal), ('t_sq_Zm_sqrt_s_hv', diff), ('t_sq_Zm_s2_hv', diff)],
        [('u_ZmDev_sq_Zm_sqrt_s_hv', gs),  ('u_ZmDev_sq_Zm_s2_hv', anal), ('', ''), ('', '')],
        [('v_ZmDev_sq_Zm_sqrt_s_hv', gs),  ('v_ZmDev_sq_Zm_s2_hv', anal), ('', ''), ('', '')],
        [('t_ZmDev_sq_Zm_sqrt_s_hv', gs),  ('t_ZmDev_sq_Zm_s2_hv', anal), ('', ''), ('', '')],
    ])
    plot_list = np.array([
    [('u_Zm', gs), ('u_Zm', anal), ('u_sq_Zm_sqrt_s_v', diff), ('kinet_Zm', diff)],
    [('v_Zm', gs), ('v_Zm', anal), ('v_sq_Zm_sqrt_s_v', diff), ('total_Zm', diff)],
    [('t_Zm', gs), ('t_Zm', anal), ('t_sq_Zm_sqrt_s_v', diff, diff), ('t2_Zm', diff)],
    [('u_ZmDev_sq_Zm_sqrt_s_v', gs),  ('u_ZmDev_sq_Zm_s2_v', anal), ('', ''), ('', '')],
    [('v_ZmDev_sq_Zm_sqrt_s_v', gs),  ('v_ZmDev_sq_Zm_s2_v', anal), ('', ''), ('', '')],
    [('t_ZmDev_sq_Zm_sqrt_s_v', gs),  ('t_ZmDev_sq_Zm_s2_v', anal), ('', ''), ('', '')],
    [('t_Zm_MmDev', gs), ('t_Zm_MmDev', anal), ('', ''), ('', '')],
    [('kinetic_Zm', gs), ('kinetic_Zm', anal), ('', ''), ('', '')],
    ])
   # [('stab_Zm', gs), ('stab_Zm', anal), ('', ''), ('', '')],

    typ_name = ['gs', 'anal', 'diff', 'diff']
    
    cbar_format = np.array([
        ['%.1f', '%.1f', '%.3f', '%.1e', '%.2f'], 
        ['%.1f', '%.1f', '%.3f', '%.1e', '%.2f'],
        ['%.1f', '%.1f', '%.3f', '%.1e', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.1e', '%.1e', '%.1e', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.2f', '%.2f', '%.2f', '%.2f', '%.1f'],
        ['%.1e', '%.1e', '%.1e', '%.2f', '%.1f'],
        ['%.3f', '%.3f', '%.1e', '%.2f', '%.1f']
    ])

    cbar_skip = np.array([
        [999, 999, 999, 2, 999, 999], 
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999],
        [999, 999, 999, 999, 999, 999]        
    ])


    cmap = np.array([
        [colormap_div, colormap_div,  colormap_seq, colormap_seq, colormap_seq], 
        [colormap_div, colormap_div,  colormap_seq, colormap_seq, colormap_seq],
        [colormap_div, colormap_div,  colormap_seq, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_seq, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_div, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_div, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_div, colormap_seq, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq, colormap_seq],
        [colormap_seq, colormap_seq, colormap_div, colormap_seq, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq, colormap_seq],
    ])

    vort_range = np.arange(-1e-4, 1.2e-4, 0.2e-4)
    vort_range = np.where(abs(vort_range) < 2e-20, 0.0, vort_range)
    conlevels = np.array([
        [np.arange(-10, 150, 10), np.arange(-50, 150, 10), np.arange(0, 14, 1), np.arange(0, 14, 1)],
        [np.arange(-10, 12, 2), np.arange(-25, 30, 5), np.arange(0, 14, 1), np.arange(0, 14, 1)],
        [np.arange(-30, 35,  5), np.arange(-30, 35,  5), np.arange(0, 3, 0.2), np.arange(0, 3, 0.2)],
        [np.arange(-30, 35,  5), np.arange(-30, 35,  5), np.arange(0, 3, 0.2), np.arange(0, 3, 0.2)],
        [np.arange(0, 14, 1), np.arange(0, 14, 1), '', ''], 
        [np.arange(0, 14, 1), np.arange(0, 14, 1), '', ''], 
        [np.arange(0, 3, 0.2), np.arange(0, 3, 0.2), '', ''] ,       
        [np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1)],
        [np.arange(-12, 13,  1), np.arange(-12, 13,  1), np.arange(-5, 5.5, 0.5), np.arange(0, 1600, 100)],
        [np.arange(-45, 50, 5), np.arange(-45, 50, 5), np.arange(-10, 11, 1), np.arange(0, 1e4, 500)],
        
    ])

    print('Height Lat  \n')
    pdf_file = path_out +'/height-lat-TimeEvolution' + sdate_plot + '-' + edate_plot + '.pdf'
    with PdfPages(pdf_file) as pdf: 
        for time in plot_index:
            print('working on time ', time_labels[time])
            fig, axes = plt.subplots(len(plot_list), 4, figsize=(20 * len(plot_list[0]), 20*len(plot_list)), sharex=False)
            for i in range(len(plot_list)):
                for j in range(len(plot_list[i])):
                    ax = axes[i, j]  
                    var = plot_list[i, j][0]
                    typ = plot_list[i, j][1]
                    if var == '':
                        continue
                    data = typ[var][time, si_h: ei_h+1, :, 0]
                    if set_levels[i, j]:
                        fills = ax.contourf(data, cmap=cmap[i, j], levels=conlevels[i, j], extend='both')
                    else:
                        if cmap[i, j] == colormap_div:
                            fills = ax.contourf(data, cmap=cmap[i, j], norm=colors.CenteredNorm(), extend='both')
                        else:
                            fills = ax.contourf(data, cmap=cmap[i, j], extend='max') # norm=colors.LogNorm(vmin=data.min(), vmax=data.max())
                    cbar = fig.colorbar(fills, orientation="horizontal", ax=ax,  format=cbar_format[i, j])
                    if cbar_skip[i, j] != 999:
                        for label in cbar.ax.xaxis.get_ticklabels()[::cbar_skip[i, j]]:
                            label.set_visible(False)                            
                    ax.set_title( typ_name[j] + ' \n ' + var, pad = 40)     
                    ax.set_xlabel('Latitude')
                    ax.set_ylabel('Height (km)')
                    ax.set_xticks(lat_ticks)
                    ax.set_xticklabels(lat_ticklabels, rotation=45)        
                    ax.set_yticks(h_ticks)
                    ax.set_yticklabels(h_ticklabels)                 
            fig.tight_layout()
            # plt.legend(loc='upper left',  bbox_to_anchor=(1, 0.5))
            fig.suptitle('rnorm-' + str(rnorm) + '-rhr-' + str(rhr) + ' \n'+
                         'Zm  ' +  'time  ' + str(time_labels[time]) )
            fig.subplots_adjust(top=0.95)
            pdf.savefig(bbox_inches = 'tight') 
            if plot_on_nb: 
                plt.show()
            plt.close(fig)
        
end_time = datetime.now()
print(end_time - start_time)

#### Height-Lon time evolution

In [ ]:
path_out

In [ ]:
np.arange(-1, 1.2, 0.2)

In [ ]:
# Create a custom formatter to replace -0.0 with 0.0
def label_formatter(x, pos):
    if x == -0.0:
        return '0.0'
    else:
        return str(x)

In [ ]:
np.arange(-0.2, 0.25,  0.05)

In [ ]:
if height_lon_plot:
    start_time = datetime.now()

    overlay = False
    lats_plot = [0, 20, 45, 65, 80]
    lats_plot = [0, 45, 60, 65, 75,  80]
    lats_plot = [0, 45, 80]
    plot_list = np.array([
        [('u_ZmDev_s_v', gs), ('u_ZmDev_s_v', anal), ('u_s_v', diff), ('u_abs_s_v', diff)],
        [('v_ZmDev_s_v', gs), ('v_ZmDev_s_v', anal), ('v_s_v', diff), ('v_abs_s_v', diff)],
        [('t_ZmDev_s_v', gs), ('t_ZmDev_s_v', anal), ('t_s_v', diff), ('t_abs_s_v', diff)],
       
    ])
     #[('stab', gs), ('stab', anal), ('stab', diff), ('', '')],
            #[('heatf_s2_v', gs), ('heatf_s2_v', anal), ('heatf_s2_v', diff), ('', '')],
       # [('momf_s2_v', gs), ('momf_s2_v', anal), ('momf_s2_v', diff), ('', '')],
        #[('vort_s_v', gs), ('vort_s_v', anal), ('vort_s_v', diff), ('', '')]
    
    typ_name = ['gs', 'anal', 'diff', 'diff']
    
    cbar_format = np.array([
        ['%.2f', '%.2f', '%.2f', '%.2f'], 
        ['%.2f', '%.2f', '%.2f', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f'],
        ['%.1e', '%.1e', '%.1e', '%.1e'],
        ['%.2f', '%.2f', '%.2f', '%.2f'],
        ['%.2f', '%.2f', '%.2f', '%.2f']       
    ])

    cbar_skip = np.array([
        [999, 999, 999, 999], 
        [999, 999, 9999, 999],
        [2, 999, 2, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [2, 2, 2, 2],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999]        
    ])


    cmap = np.array([
        [colormap_div, colormap_div, colormap_div, colormap_seq], 
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq]        
    ])

    set_levels = np.array([
        [True, False, True, False],
        [True, False, True, False],
        [True, False, True, False],
        [False, False, False, False],
        [False, False, False, False],      
    ])

    set_levels = np.array([
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],      
    ])
    vort_range = np.arange(-1e-4, 1.2e-4, 0.2e-4)
    vort_range = np.where(abs(vort_range) < 2e-20, 0.0, vort_range)
    conlevels= np.array([
        [np.arange(-1, 1.2, 0.2), np.arange(-50, 150, 10), np.arange(-1, 1.2, 0.2), np.arange(0, 14, 1)],
        [np.arange(-1, 1.2, 0.2), np.arange(-25, 30, 5), np.arange(-1, 1.2, 0.2), np.arange(0, 14, 1)],
        [np.arange(-0.2, 0.25,  0.05), np.arange(-30, 35,  5), np.arange(-0.2, 0.25,  0.05), np.arange(0, 3, 0.2)],
    ])

    print('cross section Height-Lon \n')
    for lat in lats_plot:
        lat_index = np.argmin(np.abs(lats-lat))
        print('cross section at latitude: ', lats[lat_index])
        pdf_file = path_out +'/height-lon-At-lats' + str(lat) + '-' + sdate_plot + '-' + edate_plot + '.pdf'
        with PdfPages(pdf_file) as pdf: 
            for time in plot_index:
                print('working on time ', time_labels[time])
                fig, axes = plt.subplots(len(plot_list), 4, figsize=(20 * len(plot_list[0]), 20*len(plot_list)), sharex=False)
                for i in range(len(plot_list)):
                    for j in range(len(plot_list[i])):
                        ax = axes[i, j]  
                        var = plot_list[i, j][0]
                        typ = plot_list[i, j][1]
                        if var == '':
                            continue
                        data = typ[var][time, si_h: ei_h+1, lat_index, :]
                        if set_levels[i, j]:
                            fills = ax.contourf(data, cmap=cmap[i, j], levels=conlevels[i, j], extend='both')
                        else:
                            if cmap[i, j] == colormap_div:
                                fills = ax.contourf(data, cmap=cmap[i, j], norm=colors.CenteredNorm(), extend='both')    
                            else: 
                                fills = ax.contourf(data, cmap=cmap[i, j], extend='max') 
                        cbar = fig.colorbar(fills, orientation="horizontal", ax=ax,  
                                            format=cbar_format[i, j], pad=0.2)
                        cbar.ax.tick_params(labelsize=50)
                        #cbar.ax.xaxis.set_major_formatter(FuncFormatter(label_formatter))
                        #cbar.ax.set_xticklabels([label_formatter(label.get_position(), None) for label in cbar.ax.get_xticklabels()])
                        
                        if cbar_skip[i, j] != 999:
                            for label in cbar.ax.xaxis.get_ticklabels()[::cbar_skip[i, j]]:
                                label.set_visible(False) 
                                
                        if overlay: 
                            if  var == 't_ZmDev_s_v':
                                if lat == 0:
                                    CS = ax.contour(typ['u_ZmDev_s_v'][time, si_h: ei_h+1, lat_index, :], linewidths=3, colors='black') 
                                else:    
                                    CS = ax.contour(typ['v_ZmDev_s_v'][time, si_h: ei_h+1, lat_index, :], linewidths=3, colors='black') 
                            elif var == 't_s_v': 
                                if lat == 0:
                                    CS = ax.contour(typ['u_s_v'][time, si_h: ei_h+1, lat_index, :], linewidths=3, colors='black')
                                else:   
                                    CS = ax.contour(typ['v_s_v'][time, si_h: ei_h+1, lat_index, :], linewidths=3, colors='black')

                        ax.set_title(typ_name[j] + ' \n' + var , pad = 40, fontsize=40)    
                        ax.set_ylabel('Height (km)', fontsize=50)
                        ax.set_xlabel('Longitude', fontsize=50)   
                        ax.set_xticks(lon_ticks)
                        ax.set_xticklabels(lon_ticklabels, rotation=45, fontsize=50)        
                        ax.set_yticks(h_ticks)
                        ax.set_yticklabels(h_ticklabels, fontsize=50) 
                        #ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
                    #fig.tight_layout()
                    # plt.legend(loc='upper left',  bbox_to_anchor=(1, 0.5))
                fig.suptitle('rnorm-' + str(rnorm) + '-rhr-' + str(rhr) + ' \n'+
                             'Latitude: ' + str(lat) + '\n' + 'time: '   + str(time_labels[time]), fontsize=50)
                fig.subplots_adjust(top=0.9)
                pdf.savefig(bbox_inches = 'tight')
                if plot_on_nb:
                    plt.show()
                plt.close(fig)
end_time = datetime.now()
print(end_time - start_time)                

#### Horizontal plot time evolution

In [ ]:
if lat_lon_plot: 

    start_time = datetime.now()

    hs = [80, 75, 70, 65, 60, 55, 54, 50, 45, 40]
    hs = [40, 47, 50, 52, 55, 60, 70, 80, 100]
    #hs  = [70]
    intvx = 1
    intvy = 1
    #?
    
    # if you don't want to plot, need to place a place holder in the array ('', '')
    plot_list = np.array([
        [('u_ZmDev', gs), ('u_ZmDev', anal), ('u', diff), ('u_abs', diff)],
        [('v_ZmDev', gs), ('v_ZmDev', anal), ('v', diff), ('v_abs', diff)],
        [('t_ZmDev', gs), ('t_ZmDev', anal), ('t', diff), ('t_abs', diff)],
        
    ])
    #[('t_ZmDev', gs), ('t_ZmDev', anal), ('t', diff), ('t_abs', diff)]

    # [('vort', gs), ('vort', anal), ('', ''), ('', '')]
    typ_name = ['gs', 'anal', 'diff', 'diff']
    
    cbar_format = np.array([
        ['%.1f', '%.1f', '%.1f', '%.1f'], 
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1e', '%.1e', '%.1e', '%.1e'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f']
    ])

    cbar_skip = np.array([
        [999, 999, 999, 999], 
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [2, 2, 2, 2],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999]
    ])


    cmap = np.array([
        [colormap_div, colormap_div, colormap_div, colormap_seq], 
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq]
    ])

    set_levels = np.array([
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]
    ])

    vort_range = np.arange(-1e-4, 1.2e-4, 0.2e-4)
    vort_range = np.where(abs(vort_range) < 2e-20, 0.0, vort_range)
    conlevels = np.array([
        [np.arange(-50, 50, 10), np.arange(-50, 150, 10), 0.005*np.arange(-1, 1.1, 0.1), 0.005*np.arange(-1, 1.1, 0.1)],
        [np.arange(-25, 30, 5), np.arange(-25, 30, 5), 0.005*np.arange(-1, 1.1, 0.1), 0.005*np.arange(-1, 1.1, 0.1)]
    ])

    intvx = 1    # ? 2 is not working yet
    intvy = 1

    print('horizontal plot \n')
    for h in hs:
        lev = np.argmin(np.abs(heights - h))
        print('height', h)
        pdf_file = path_out +'/horizontal-' + str(h) + 'km-' + sdate_plot + '-' + edate_plot + '.pdf'
        with PdfPages(pdf_file) as pdf: 
            for time in plot_index:  
                print('working on time ', time_labels[time])
                fig, axes = plt.subplots(len(plot_list), 4, figsize=(20 * len(plot_list[0]), 20*len(plot_list)), sharex=False)
                for i in range(len(plot_list)):
                    #print('working on variable', var)
                    for j in range(len(plot_list[i])):
                        ax = axes[i, j]
                        var = plot_list[i, j][0]
                        typ = plot_list[i, j][1]
                        if var == '':
                            continue
                        if set_levels[i, j]:
                            fills = ax.contourf(typ[var][time, lev, ::intvy, ::intvx],
                                                cmap=cmap[i, j], levels=conlevels[i, j], extend='max')
                        else:
                            if cmap[i, j] == colormap_div:
                                fills = ax.contourf(typ[var][time, lev, ::intvy, ::intvx], 
                                                    cmap=cmap[i, j], norm=colors.CenteredNorm(), extend='both')
                            else:
                                fills = ax.contourf(typ[var][time, lev, ::intvy, ::intvx], 
                                                    cmap=cmap[i, j], extend='max')

                        if i == 3:  
                            Q = ax.quiver(typ['u_ZmDev'][time, lev, ::intvy, ::intvx], 
                                      typ['v_ZmDev'][time, lev, ::intvy, ::intvx] , 
                                   scale=20, scale_units='inches') 
                            quiverkey_length = 2
                            # I don't know how to set the quiverkey yet, do it later
                            qk = ax.quiverkey(Q, 0.9, 1.05, 
                                              quiverkey_length, str(quiverkey_length) +  'm/s', 
                                              labelpos='E', coordinates='axes')
                        ax.set_title(var + '_' + typ_name[j] + '  at height ' + str(h) + ' km'
                                    , pad = 40, fontsize=60)
                        cbar = fig.colorbar(fills, orientation="horizontal", ax=ax,  format=cbar_format[i, j], pad=0.2)
                        cbar.ax.tick_params(labelsize=60)
                        if cbar_skip[i, j] != 999:
                            for label in cbar.ax.xaxis.get_ticklabels()[::cbar_skip[i, j]]:
                                label.set_visible(False)

                        ax.set_ylabel('Latitude', fontsize=60)
                        ax.set_xlabel('Longitude', fontsize=60)
                        ax.set_xticks(lon_ticks[::intvx])
                        ax.set_xticklabels(lon_ticklabels[::intvx], rotation=45, fontsize=60)     
                        ax.set_yticks(lat_ticks[::intvy])
                        ax.set_yticklabels(lat_ticklabels[::intvy], fontsize=60) 
                        ax.tick_params(axis='both', which='major', length=20, width=8)
                fig.tight_layout()
                #plt.legend(loc='upper left',  bbox_to_anchor=(1, 0.5))
                fig.suptitle('rnorm-' + str(rnorm) + '-rhr-' + str(rhr) + ' \n'+
                             'time  ' + str(time_labels[time]), fontsize=60)
                fig.subplots_adjust(top=0.9)
                pdf.savefig(bbox_inches = 'tight') 
                if plot_on_nb:
                    plt.show()
                plt.close(fig)   
end_time = datetime.now()
print(end_time - start_time)                

In [ ]:
path_out

## composite mean

In [ ]:
time_labels

In [ ]:
print('prepare the data for composite plot')

time_labels_composite = generate_time_labels(sdate_comp, edate_comp, dm=0, dday=0, dh=dh_plot)
time_labels_composite

In [ ]:
plot_index_composite = time_labels[time_labels.isin(time_labels_composite)].index
print(plot_index_composite)

#### concatinate the data along the longitude

In [ ]:
time_labels_composite

In [ ]:
if composite_plot:
    vars_comp = ['u',  'u_ZmDev', 'u_sq',  'v',  'v_ZmDev', 'v_sq', 't', 't_ZmDev', 't_sq']
    for i, typ in enumerate([gs, anal, diff]):
        for var in vars_comp:
            print(var)
            # skip these variables in gs and anal 
            if ((i == 0) | (i == 1)) & (var in ['u_sq', 'v_sq', 't_sq']):
                continue
                
            data = typ[var][plot_index_composite]
            concat = np.concatenate([data, data, data], axis=3)
            
            # center the plots
            dims_composite = data.shape
            center = np.zeros(dims_composite)
            for time_index, time_label in enumerate(time_labels_composite):
                #print(time_label)
                solar_center_i = qsol.loc[time_label][0]
                #print('solar_center_i', solar_center_i)
                #if sup_composition:
                #    solar_center_i = sup_i[time_label]
                    #print('super rotation position', solar_center_i)       
                mi = nlon + solar_center_i  
                si = mi - int(nlon/2) + 1
                ei = mi + int(nlon/2)
                center[time_index] = concat[time_index, :, :, si:ei+1]
                
            # composite mean
            comp = center.mean(axis=0, dtype=np.float64)
            #comp_sqrt = np.sqrt(comp)
            
            # scale the data
            comp_s_v = comp * s_v[0]  # comp is 3D
            comp_s_h = comp * s_h[0]
            comp_s2_v = comp * s2_v[0]
            comp_s2_h = comp * s2_h[0]
            typ[var+'_comp'] = comp
            typ[var+'_comp_s_v'] = comp_s_v
            typ[var+'_comp_s_h'] = comp_s_h
            typ[var+'_comp_s2_v'] = comp_s2_v
            typ[var+'_comp_s2_h'] = comp_s2_h

In [ ]:
gs['u'].shape

## plot composite

#### Horizontal plot composite

In [ ]:
gs['u_Tm'].shape

In [ ]:
if composite_lat_lon_plot: 
    start_time = datetime.now()
    overlay = False
    hs = [80, 75, 70, 65, 60, 55, 54, 50, 45, 40]
    #hs = [80, 70, 60, 50]  
    #hs = [50, 60, 70, 80, 100]
    #hs = [55, 60, 65, 75, 85, 90]
    #hs = [60, 65]

    intvx = 1
    intvy = 1
    #?
    
    # if you don't want to plot, need to place a place holder in the array ('', '')
    plot_list = np.array([
        [('u_ZmDev_comp', gs), ('u_ZmDev_comp', anal), ('u_comp', diff), ('u_sq_comp', diff)],
        [('v_ZmDev_comp', gs), ('v_ZmDev_comp', anal), ('v_comp', diff), ('v_sq_comp', diff)],
        [('t_ZmDev_comp', gs), ('t_ZmDev_comp', anal), ('t_comp', diff), ('t_sq_comp', diff)],
    ])

    typ_name = ['gs', 'anal', 'diff', 'diff']
    
    cbar_format = np.array([
        ['%.1f', '%.1f', '%.0f', '%.1f'], 
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.2f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f']
    ])

    cbar_skip = np.array([
        [999, 999, 999, 2], 
        [999, 999, 999, 2],
        [999, 999, 999, 2],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999]
    ])


    cmap = np.array([
        [colormap_div, colormap_div, colormap_div, colormap_seq], 
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq]
    ])

    set_levels = np.array([
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]
    ])

    vort_range = np.arange(-1e-4, 1.2e-4, 0.2e-4)
    vort_range = np.where(abs(vort_range) < 2e-20, 0.0, vort_range)
    conlevels = np.array([
        [np.arange(-50, 50, 10), np.arange(-50, 150, 10), 0.005*np.arange(-1, 1.1, 0.1), 0.005*np.arange(-1, 1.1, 0.1)],
        [np.arange(-25, 30, 5), np.arange(-25, 30, 5), 0.005*np.arange(-1, 1.1, 0.1), 0.005*np.arange(-1, 1.1, 0.1)]
    ])

    intvx = 1    # ? 2 is not working yet
    intvy = 1

    print('horizontal plot \n')
    for h in hs:
        lev = np.argmin(np.abs(heights - h))
        print('height', h)
        pdf_file = path_out +'/comp-horizontal-' + str(h) + 'km-' + sdate_comp + '-' + edate_comp + '.pdf'
        with PdfPages(pdf_file) as pdf: 
            fig, axes = plt.subplots(len(plot_list), 4, figsize=(20 * len(plot_list[0]), 20*len(plot_list)), sharex=False)
            for i in range(len(plot_list)):
                #print('working on variable', var)
                for j in range(len(plot_list[i])):
                    ax = axes[i, j]
                    var = plot_list[i, j][0]
                    typ = plot_list[i, j][1]
                    if var == '':
                        continue
                    if set_levels[i, j]:
                        fills = ax.contourf(typ[var][lev, ::intvy, ::intvx],
                                            cmap=cmap[i, j], levels=conlevels[i, j], extend='max')
                    else:
                        if cmap[i, j] == colormap_div:
                            fills = ax.contourf(typ[var][lev, ::intvy, ::intvx], 
                                                cmap=cmap[i, j], norm=colors.CenteredNorm(), extend='both')
                        else:
                            fills = ax.contourf(typ[var][lev, ::intvy, ::intvx], 
                                                cmap=cmap[i, j], extend='max')

                    if overlay:
                        if var == 'u_ZmDev_comp':
                            CS = ax.contour(typ['u_Tm'][0, lev, ::intvy, ::intvx], linewidths=3, colors='black') 
                        elif var == 'v_ZmDev_comp':
                            CS = ax.contour(typ['v_Tm'][0, lev, ::intvy, ::intvx], linewidths=3, colors='black')                         
                        elif var == 't_ZmDev_comp':
                            CS = ax.contour(typ['t_Tm'][0, lev, ::intvy, ::intvx], linewidths=3, colors='black') 
  
                    ax.set_title(typ_name[j] + ' \n' +  var +  '  at height ' + str(h) + ' km'
                                , pad = 40, fontsize=60)
                    cbar = fig.colorbar(fills, orientation="horizontal", ax=ax,  format=cbar_format[i, j], pad=0.22)
                    cbar.ax.tick_params(labelsize=60)
                    if cbar_skip[i, j] != 999:
                        for label in cbar.ax.xaxis.get_ticklabels()[::cbar_skip[i, j]]:
                            label.set_visible(False)

                    ax.set_ylabel('Latitude', fontsize=55)
                    ax.set_xlabel('Longitude', fontsize=60)
                    ax.set_xticks(lon_ticks[::intvx])
                    ax.set_xticklabels(lon_ticklabels[::intvx], rotation=45, fontsize=60)     
                    ax.set_yticks(lat_ticks[::intvy])
                    ax.set_yticklabels(lat_ticklabels[::intvy], fontsize=60) 
                    ax.tick_params(axis='both', which='major', length=20, width=8)
            fig.tight_layout()
            #plt.legend(loc='upper left',  bbox_to_anchor=(1, 0.5))
            fig.suptitle('composite rnorm-' + str(rnorm) + '-rhr-' + str(rhr), fontsize=50 )
            fig.subplots_adjust(top=0.90)
            pdf.savefig(bbox_inches = 'tight') 
            if plot_on_nb:
                plt.show()
            plt.close(fig)   
end_time = datetime.now()
print(end_time - start_time)                

#### Height-Lon composite

In [ ]:
if composite_height_lon_plot:
    start_time = datetime.now()

    overlay = False
    lats_plot = [0, 20, 45, 60, 65, 75,  80]
    #lats_plot = [45]
    
    
    plot_list = np.array([
        [('u_ZmDev_comp_s_v', gs), ('u_ZmDev_comp_s_v', anal), ('u_comp_s_v', diff), ('u_sq_comp_s2_v', diff)],
        [('v_ZmDev_comp_s_v', gs), ('v_ZmDev_comp_s_v', anal), ('v_comp_s_v', diff), ('v_sq_comp_s2_v', diff)],
        [('t_ZmDev_comp_s_v', gs), ('t_ZmDev_comp_s_v', anal), ('t_comp_s_v', diff), ('t_sq_comp_s2_v', diff)],
    ])

    typ_name = ['gs', 'anal', 'diff', 'diff']
    
    cbar_format = np.array([
        ['%.2f', '%.2f', '%.1e', '%.1e'], 
        ['%.2f', '%.2f', '%.1e', '%.1e'],
        ['%.2f', '%.2f', '%.1e', '%.1e'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f'],
        ['%.1f', '%.1f', '%.1f', '%.1f']        
    ])

    cbar_skip = np.array([
        [2, 999, 2, 2], 
        [2, 999, 2, 2],
        [2, 999, 2, 2],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999],
        [999, 999, 999, 999]        
    ])


    cmap = np.array([
        [colormap_div, colormap_div, colormap_div, colormap_seq], 
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq],
        [colormap_div, colormap_div, colormap_div, colormap_seq]        
    ])

    set_levels = np.array([
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]        
    ])

    vort_range = np.arange(-1e-4, 1.2e-4, 0.2e-4)
    vort_range = np.where(abs(vort_range) < 2e-20, 0.0, vort_range)
    conlevels = np.array([
        [np.arange(-10, 150, 10), np.arange(-50, 150, 10), np.arange(0, 14, 1), np.arange(0, 14, 1)],
        [np.arange(-10, 12, 2), np.arange(-25, 30, 5), np.arange(0, 14, 1), np.arange(0, 14, 1)],
        [np.arange(-30, 35,  5), np.arange(-30, 35,  5), np.arange(0, 3, 0.2), np.arange(0, 3, 0.2)],
        [np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1), np.arange(0, 1, 0.1)],
        [np.arange(-12, 13,  1), np.arange(-12, 13,  1), np.arange(-5, 5.5, 0.5), np.arange(0, 1600, 100)],
        [np.arange(-45, 50, 5), np.arange(-45, 50, 5), np.arange(-10, 11, 1), np.arange(0, 1e4, 500)],
        [np.arange(0, 14, 1), np.arange(0, 14, 1), '', ''],
        [np.arange(0, 14, 1), np.arange(0, 14, 1), '', ''],
        [np.arange(0, 14, 1), np.arange(0, 14, 1), '', '']
    ])

    print('cross section Height-Lon \n')
    for lat in lats_plot:
        lat_index = np.argmin(np.abs(lats-lat))
        print('cross section at latitude: ', lats[lat_index])
        pdf_file = path_out +'/comp-height-lon-At-lats' + str(lat) + '-' + sdate_comp + '-' + edate_comp + '.pdf'
        with PdfPages(pdf_file) as pdf: 
            fig, axes = plt.subplots(len(plot_list), 4, figsize=(22 * len(plot_list[0]), 22 * len(plot_list)), sharex=False)
            for i in range(len(plot_list)):
                for j in range(len(plot_list[i])):
                    ax = axes[i, j]  
                    var = plot_list[i, j][0]
                    typ = plot_list[i, j][1]
                    if var == '':
                        continue
                    data = typ[var][si_h: ei_h+1, lat_index, :]
                    if set_levels[i, j]:
                        fills = ax.contourf(data, cmap=cmap[i, j], levels=conlevels[i, j], extend='max')
                    else:
                        if cmap[i, j] == colormap_div:
                            fills = ax.contourf(data, cmap=cmap[i, j], norm=colors.CenteredNorm(), extend='both')    
                        else: 
                            fills = ax.contourf(data, cmap=cmap[i, j], extend='max') 
                    cbar = fig.colorbar(fills, orientation="horizontal", ax=ax,  format=cbar_format[i, j], pad=0.25)
                    cbar.ax.tick_params(labelsize=60)
                    if cbar_skip[i, j] != 999:
                        for label in cbar.ax.xaxis.get_ticklabels()[::cbar_skip[i, j]]:
                            label.set_visible(False)                      
                    if overlay:
                        if var == 't_ZmDev_comp_s_v':
                            CS = ax.contour(typ['v_ZmDev_comp_s_v'][si_h: ei_h+1, lat_index, :], linewidths=3, colors='black') 
                        elif var == 't_comp_s_v':
                            CS = ax.contour(typ['v_comp_s_v'][si_h: ei_h+1, lat_index, :], linewidths=3, colors='black')                           

                    ax.set_title(typ_name[j] + ' \n' + var , pad = 40, fontsize=60)    
                    ax.set_ylabel('Height (km)', fontsize=60)
                    ax.set_xlabel('Longitude', fontsize=60)   
                    ax.set_xticks(lon_ticks)
                    ax.set_xticklabels(lon_ticklabels, rotation=45, fontsize=60)        
                    ax.set_yticks(h_ticks)
                    ax.set_yticklabels(h_ticklabels, fontsize=60) 
                    ax.tick_params(axis='both', which='major', length=20, width=8)
                    #ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
                #fig.tight_layout()
                # plt.legend(loc='upper left',  bbox_to_anchor=(1, 0.5))
            fig.suptitle('composite' + '\n' + 
                         'rnorm-' + str(rnorm) + '-rhr-' + str(rhr) + ' \n' +
                         'Latitude: ' + str(lat),  fontsize=60)
            fig.subplots_adjust(top=0.9)
            pdf.savefig(bbox_inches = 'tight')
            if plot_on_nb:
                plt.show()
            plt.close(fig)
end_time = datetime.now()
print(end_time - start_time)                

In [ ]:
path_out